In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
#Đọc file dữ liệu crawl được từ trang batdongsan.com.vn
d1 = pd.read_csv('final_data_0-1999.csv')
d2 = pd.read_csv('final_data 2000-4000.csv')
d3 = pd.read_csv('final_data_4000-6000.csv')
d4 = pd.read_csv('final_data_10000.csv')
d5 = pd.read_csv('final_data_20000.csv')

In [3]:
#Gộp tất cả data lại với nhau
df = pd.concat([d1, d2, d3, d4, d5], ignore_index = True)

#Loại bỏ các hàng trùng nhau
df = df.drop_duplicates()

# Đánh số thứ tự lại các dòng
df.reset_index(drop=True, inplace=True)

#:Loại bỏ các cột không cần thiết
columns_to_drop = ['Mặt tiền', 'Đường vào', 'Hướng nhà', 'Quận', 'Hướng ban công', 'Địa chỉ', 'Số tầng', 'Pháp lý', 'Ngày đăng']
df = df.drop(columns=columns_to_drop)

# Đổi tên cột 'Thành phố' thành 'Quận' (do sai sót khi crawl)
df = df.rename(columns={'Thành phố': 'Quận'})
df = df.rename(columns={'Diện tích': 'Diện tích (m²)'})
df = df.rename(columns={'Mức giá': 'Mức giá (tỷ)'})

# Sắp xếp lại thứ tự các cột
desired_columns_order = ['Quận', 'Loại', 'Diện tích (m²)', 'Mức giá (tỷ)', 'Số phòng ngủ', 'Số toilet']
df = df[desired_columns_order]

# Loại bỏ tất cả các dòng có nội dung là "Thỏa thuận" trong cột "Mức giá"
df = df[df['Mức giá (tỷ)'] != 'Thỏa thuận']

# Loại bỏ tất cả các dòng không có nội dung trong cột "Số phòng ngủ" và "Số toilet"
df = df.dropna(subset=['Số phòng ngủ', 'Số toilet'])

# Sử dụng biểu thức chính quy để tách số từ chuỗi và chuyển nó thành số thực
df['Diện tích (m²)'] = df['Diện tích (m²)'].str.extract('(\d+)').astype(float)
df['Số phòng ngủ'] = df['Số phòng ngủ'].str.extract('(\d+)').astype(int)
df['Số toilet'] = df['Số toilet'].str.extract('(\d+)').astype(int)

df

,Quận,Loại,Diện tích (m²),Mức giá (tỷ),Số phòng ngủ,Số toilet
0,Gia Lâm,Căn hộ chung cư tại Vinhomes Ocean Park Gia Lâm,55.0,"2,07 tỷ",2,1
1,Nam Từ Liêm,Căn hộ chung cư tại Lumiere EverGreen,140.0,"9,5 tỷ",4,3
2,Hà Đông,Căn hộ chung cư tại Grand Sunlake,98.0,"3,19 tỷ",3,2
3,Nam Từ Liêm,Căn hộ chung cư tại Imperia Sola Park,59.0,"2,9 tỷ",2,2
4,Hà Đông,Căn hộ chung cư tại Grand Sunlake,92.0,"3,5 tỷ",2,2
...,...,...,...,...,...,...
11377,Bắc Từ Liêm,Căn hộ chung cư tại Sunshine City,98.0,"6,35 tỷ",3,2
11378,Hà Đông,Nhà mặt phố tại Khu đô thị mới Văn Quán,225.0,66 tỷ,6,8
11379,Nam Từ Liêm,Nhà riêng tại đường Nhổn,40.0,4 tỷ,4,4
11380,Gia Lâm,Căn hộ chung cư tại Vinhomes Ocean Park Gia Lâm,70.0,"2,45 tỷ",2,2


In [4]:
# Xác định các dòng có nội dung dạng "y triệu/m²"
y_million_per_sqm = df['Mức giá (tỷ)'].str.contains('triệu/m²')

# Thay thế dấu phẩy bằng dấu chấm và chuyển thành float
y_values = df.loc[y_million_per_sqm, 'Mức giá (tỷ)'].str.replace(',', '.').str.split().str[0].astype(float)

# Lấy diện tích từ cột "Diện tích (m²)"
areas = df.loc[y_million_per_sqm, 'Diện tích (m²)']

# Nhân y với diện tích và chuyển thành dạng tỷ
new_values = (y_values * areas / 10**3).round(2)

# Sử dụng biểu thức chính quy để tách số từ chuỗi và chuyển nó thành số thực
df['Mức giá (tỷ)'] = df['Mức giá (tỷ)'].str.extract('(\d+)').astype(float)

# Cập nhật các giá trị mới vào cột "Mức giá (tỷ)"
df.loc[y_million_per_sqm, 'Mức giá (tỷ)'] = new_values

# Loại bỏ phần "tại" và những ký tự sau đó trong cột "Loại"
df['Loại'] = df['Loại'].str.replace(r'\s*tại.*', '', regex=True)

# Đánh số lại các dòng
df = df.reset_index(drop=True)

df

,Quận,Loại,Diện tích (m²),Mức giá (tỷ),Số phòng ngủ,Số toilet
0,Gia Lâm,Căn hộ chung cư,55.0,2.0,2,1
1,Nam Từ Liêm,Căn hộ chung cư,140.0,9.0,4,3
2,Hà Đông,Căn hộ chung cư,98.0,3.0,3,2
3,Nam Từ Liêm,Căn hộ chung cư,59.0,2.0,2,2
4,Hà Đông,Căn hộ chung cư,92.0,3.0,2,2
...,...,...,...,...,...,...
7891,Bắc Từ Liêm,Căn hộ chung cư,98.0,6.0,3,2
7892,Hà Đông,Nhà mặt phố,225.0,66.0,6,8
7893,Nam Từ Liêm,Nhà riêng,40.0,4.0,4,4
7894,Gia Lâm,Căn hộ chung cư,70.0,2.0,2,2


In [5]:
# Kiểm tra nếu file "final_data.csv" đã tồn tại, thì thay thế nó bằng file mới
if os.path.exists("final_data.csv"):
    os.remove("final_data.csv")

# Lưu dataframe vào file CSV
df.to_csv('final_data.csv', index=False)

print("Dữ liệu đã được lưu vào file final_data.csv thành công.")

Dữ liệu đã được lưu vào file final_data.csv thành công.
